# Project work in KIA (2024-FS)
Notebook 1: Data Preparation </br>
Student: Vion Hasaj, hasvio01@students.zhaw.ch </br>

# Table of content

[1 Libraries and settings](#section_1)<br>
[2 Import data](#section_2)<br>
[3 Clean up scraped Data](#section_3)<br>
[4 Remove unnecessary data](#section_4)<br>


# <font color='#AAD9BB'>1 Libraries and settings </font>
<a id='section_1'></a>

In [21]:
# Libraries
import os
import re
#import time
import fnmatch
import numpy as np
import pandas as pd
#import pdfplumber
from tabula import read_pdf
from datetime import datetime

import mysql.connector
from sqlalchemy import create_engine
import json

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

c:\Users\vionh\workspace\KIAProject\data collection and preparation


# <font color='#AAD9BB'>2 Import data </font>
<a id='section_2'></a>

In [22]:
matchesWithBettingsOdds = 'data/SWZ.csv'
swz_df = pd.read_csv(matchesWithBettingsOdds)

matchesWithStats = 'data/football.csv'
football_df = pd.read_csv(matchesWithStats)

# Show first records of data frame
swz_df.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Switzerland,Super League,2016/2017,23/07/2016,16:45,Lugano,Luzern,1,2,A,3.53,3.81,2.09,3.67,3.82,2.10,3.38,3.60,2.01
1,Switzerland,Super League,2016/2017,23/07/2016,19:00,St. Gallen,Young Boys,0,2,A,4.64,4.10,1.77,4.65,4.26,1.90,4.19,3.88,1.75
2,Switzerland,Super League,2016/2017,24/07/2016,12:45,Grasshoppers,Lausanne,2,0,H,1.83,3.91,4.51,1.99,4.00,4.51,1.79,3.74,4.11
3,Switzerland,Super League,2016/2017,24/07/2016,12:45,Thun,Vaduz,1,1,D,2.20,3.46,3.55,2.20,3.58,3.80,2.09,3.37,3.41
4,Switzerland,Super League,2016/2017,24/07/2016,15:00,Basel,Sion,3,0,H,1.75,3.86,5.16,1.75,4.10,5.45,1.68,3.81,4.74


In [23]:
football_df.head()

,web-scraper-order,web-scraper-start-url,navigation,scoreLink,scoreLink-href,HTeam,ATeam,date,detail
0,1716287609-1,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑3,https://thefishy.co.uk/livematch2/fc-sion-v-ba...,FC Sion,Basel,28-Jan-2021 19:30,"[{""detail"":""5""},{""detail"":""Substitutions""},{""d..."
1,1716287611-2,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑3,https://thefishy.co.uk/livematch2/st-gallen-v-...,St Gallen,FC Zurich,30-Jan-2021 19:30,"[{""detail"":""5""},{""detail"":""Substitutions""},{""d..."
2,1716287613-3,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑1,https://thefishy.co.uk/livematch2/young-boys-v...,Young Boys,FC Sion,31-Jan-2021 15:00,"[{""detail"":""5""},{""detail"":""Substitutions""},{""d..."
3,1716287615-4,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,1‑1,https://thefishy.co.uk/livematch2/servette-fc-...,Servette FC,Lausanne Sports,31-Jan-2021 15:00,"[{""detail"":""4""},{""detail"":""Substitutions""},{""d..."
4,1716287617-5,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑2,https://thefishy.co.uk/livematch2/basel-v-fc-l...,Basel,FC Lugano,31-Jan-2021 15:00,"[{""detail"":""4""},{""detail"":""Substitutions""},{""d..."


# <font color='#AAD9BB'>3 Clean up data </font>
<a id='section_3'></a>

In [24]:
# Function to standardize the date format in football_df
def standardize_date(date_str):
    try:
        return datetime.strptime(date_str, '%d-%b-%Y %H:%M').strftime('%d/%m/%Y')
    except Exception as e:
        return None

# Revised function to process the detail column
def process_detail_column(detail):
    data = json.loads(detail)
    processed_data = {}
    
    for i in range(0, len(data), 3):
        if i + 2 < len(data):
            home_stat = data[i].get("detail", "")
            stat_description = data[i+1].get("detail", "")
            away_stat = data[i+2].get("detail", "")
            
            if stat_description:
                processed_data[f"home_{stat_description.lower().replace(' ', '_')}"] = home_stat
                processed_data[f"away_{stat_description.lower().replace(' ', '_')}"] = away_stat
                
    return processed_data

# Standardize the date format
football_df['date'] = football_df['date'].apply(standardize_date)

# Drop rows with invalid dates
football_df = football_df.dropna(subset=['date'])

# Rename the 'date' column to 'Date' to match swz_df
football_df.rename(columns={'date': 'Date'}, inplace=True)

# Rename 'Home' column in swz_df to 'HTeam' to match football_df
swz_df.rename(columns={'Home': 'HTeam'}, inplace=True)

# Create a mapping to standardize team names between the datasets
team_name_mapping = {
    'Yverdon-Sport FC': 'Yverdon',
    'FC Zurich': 'Zurich',
    'FC Winterthur': 'Winterthur',
    'Stade-Lausanne-Ouchy': 'Lausanne Ouchy',
    'St Gallen': 'St. Gallen',
    'Grasshoppers': 'Grasshoppers',
    'Servette FC': 'Servette',
    'FC Luzern': 'Luzern',
    'Young Boys': 'Young Boys',
    'Lausanne Sports': 'Lausanne',
    'Servette FC': 'Servette',
    'FC Sion': 'Sion',
    'FC Lugano': 'Lugano',
    'FC Vaduz': 'Vaduz'
}

# Apply the mapping to the 'HTeam' column in football_df
football_df['HTeam'] = football_df['HTeam'].replace(team_name_mapping)

# Apply the function to each row in the 'detail' column
processed_data_list = football_df['detail'].apply(process_detail_column)
processed_df = pd.DataFrame(processed_data_list.tolist())

# Merge the processed data back to the original dataframe
football_df = pd.concat([football_df.drop(columns=['detail']), processed_df], axis=1)

# Re-attempt the merge on 'Date' and 'HTeam' columns
merged_df = pd.merge(football_df, swz_df, on=['Date', 'HTeam'], how='inner')

merged_df['ATeam'] = merged_df['ATeam'].replace(team_name_mapping)

# Display the resulting merged DataFrame
merged_df.head()

,web-scraper-order,web-scraper-start-url,navigation,scoreLink,scoreLink-href,HTeam,ATeam,Date,home_substitutions,away_substitutions,...,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,1716287609-1,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑3,https://thefishy.co.uk/livematch2/fc-sion-v-ba...,Sion,Basel,28/01/2021,5,2,...,A,4.22,3.97,1.83,4.22,4.00,1.93,3.99,3.73,1.82
1,1716287611-2,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑3,https://thefishy.co.uk/livematch2/st-gallen-v-...,St. Gallen,Zurich,30/01/2021,5,4,...,A,2.14,3.87,3.25,2.16,3.87,3.56,2.07,3.68,3.21
2,1716287613-3,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑1,https://thefishy.co.uk/livematch2/young-boys-v...,Young Boys,Sion,31/01/2021,5,5,...,H,1.29,6.09,10.18,1.30,6.23,11.00,1.28,5.61,9.30
3,1716287615-4,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,1‑1,https://thefishy.co.uk/livematch2/servette-fc-...,Servette,Lausanne,31/01/2021,4,5,...,D,2.68,3.64,2.60,2.80,3.65,2.60,2.65,3.44,2.50
4,1716287617-5,https://thefishy.co.uk/football-results.php?ta...,https://thefishy.co.uk/football-results.php?ta...,2‑2,https://thefishy.co.uk/livematch2/basel-v-fc-l...,Basel,Lugano,31/01/2021,4,5,...,D,2.29,3.42,3.28,2.33,3.46,3.29,2.24,3.29,3.16


# <font color='#AAD9BB'>4 Remove unnecessary data </font>
<a id='section_4'></a>

In [25]:
# Drop unnecessary columns after merging
columns_to_remove = ['web-scraper-order', 'web-scraper-start-url', 'navigation','scoreLink', 'home_expected_goals', 'away_expected_goals','Country','League','PH', 'PD', 'PA', 'MaxH', 'MaxD', 'MaxA']
result_df = merged_df.drop(columns=columns_to_remove)

# Calculate the median for each column
home_offside_median = result_df['home_offsides'].median()
away_offside_median = result_df['away_offsides'].median()
home_subs_median = result_df['home_substitutions'].median()
away_subs_median = result_df['away_substitutions'].median()

# Fill NaN values with the respective median
result_df['home_offsides'].fillna(home_offside_median, inplace=True)
result_df['away_offsides'].fillna(away_offside_median, inplace=True)
result_df['home_substitutions'].fillna(home_subs_median, inplace=True)
result_df['away_substitutions'].fillna(away_subs_median, inplace=True)
result_df.rename(columns={'AvgH': 'avg_betting_odds_home'}, inplace=True)
result_df.rename(columns={'AvgD': 'avg_betting_odds_draw'}, inplace=True)
result_df.rename(columns={'AvgA': 'avg_betting_odds_away'}, inplace=True)

result_df = result_df.dropna(subset=['home_shots_blocked', 'away_shots_blocked'])

# Display the number of rows and first few rows of the final cleaned dataframe to verify the changes
num_rows_final_cleaned_df_corrected = result_df.shape[0]

# Check for NaN values in the DataFrame
nan_values_final_cleaned = result_df.isna().sum()

# Display the columns with NaN values and their count
nan_columns = nan_values_final_cleaned[nan_values_final_cleaned > 0]
print(nan_columns)


result_df['home_shots_on_target'] = pd.to_numeric(result_df['home_shots_on_target'], errors='coerce')
result_df['HG'] = pd.to_numeric(result_df['HG'], errors='coerce')
result_df['away_shots_on_target'] = pd.to_numeric(result_df['away_shots_on_target'], errors='coerce')
result_df['AG'] = pd.to_numeric(result_df['AG'], errors='coerce')

# Fill NaN values with 0 (or another appropriate value)
result_df.fillna(0, inplace=True)

# Fill 'home_shots_per_goal' column and set 'home_shots_on_target' to 0 if 'HG' is 0
result_df['home_shots_per_goal'] = result_df.apply(
    lambda row: row['home_shots_on_target'] / row['HG'] if row['HG'] != 0 else 0,
    axis=1
)

result_df['home_shots_on_target'] = result_df.apply(
    lambda row: 0 if row['HG'] == 0 else row['home_shots_on_target'],
    axis=1
)

# Fill 'away_shots_per_goal' column and set 'away_shots_on_target' to 0 if 'AG' is 0
result_df['away_shots_per_goal'] = result_df.apply(
    lambda row: row['away_shots_on_target'] / row['AG'] if row['AG'] != 0 else 0,
    axis=1
)

result_df['away_shots_on_target'] = result_df.apply(
    lambda row: 0 if row['AG'] == 0 else row['away_shots_on_target'],
    axis=1
)

result_df.to_csv("../eda/preprocessed_matches.csv")

Series([], dtype: int64)
